### **Table of Contents**
    
* [Reading in the data](#reading-in-the-data)
  * [Finished import](#finished-import)
* [cleaning](#cleaning)
* [Employee Pay and Hours Calculations](#employee-pay-and-hours-calculations)
    * [1️ Hourly Rate and Minimum Wage Adjustment](#1️-hourly-rate-and-minimum-wage-adjustment)
    * [2️ Overtime Rate](#2️-overtime-rate)
    * [3️ Regular Hours and Full-Time Filter](#3️-regular-hours-and-full-time-filter)
    * [4️ Regular and Overtime Hours Worked](#4️-regular-and-overtime-hours-worked)
    * [5️ Total Hours Worked](#5️-total-hours-worked)
    * [6️ Weeks Worked](#6️-weeks-worked)
    * [7️ Weekly Breakdown of Regular and Overtime Hours](#7️-weekly-breakdown-of-regular-and-overtime-hours)
    * [8️ Average Hours per Week](#8️-average-hours-per-week)
* [Keep this file in your project](#keep-this-file-in-your-project)


In [34]:
import pandas as pd
import numpy as np

# formatting notebook to show rounded values.
pd.options.display.float_format = '{:,.2f}'.format

# Reading in the data 

- Reading in the data 
- finding the data types for each column. 
- optimized each column to not have objects as they are more memory hungry.  

In [35]:
# sal = pd.read_csv("data/salary2.csv")

# sal.info()

# df = sal
# df.info(memory_usage='deep')
# df["Department"] = df["Department"].astype("category")
# df["jobTitle"] = df["jobTitle"].astype("category")
# df["Employee_Name"] = df["Employee_Name"].astype("string")
# df.info(memory_usage='deep')

# optimized_dtypes = {
#     "CalYear": "int16",
#     "Employee_Name": "string",
#     "Department": "category",
#     "jobTitle": "category",
#     "Annual_Rate": "float32",
#     "Regular_Rate": "float32",
#     "Overtime_Rate": "float32",
#     "Incentive_Allowance": "float32",
#     "Other": "float32",
#     "YTD_Total": "float32",
#     "ObjectId": "int32"
# }
# df = df.astype(optimized_dtypes)


## Finished import 
- now importing the data with the data types predefined 

In [36]:
optimized_dtypes = {
    "CalYear": "int16",
    "Employee_Name": "string",
    "Department": "category",
    "jobTitle": "category",
    "Annual_Rate": "float32",
    "Regular_Rate": "float32",
    "Overtime_Rate": "float32",
    "Incentive_Allowance": "float32",
    "Other": "float32",
    "YTD_Total": "float32",
    "ObjectId": "int32"
}

sal = pd.read_csv("data/salary2.csv", dtype=optimized_dtypes)


# cleaning 

In [37]:
sal.head()

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,Other,YTD_Total,ObjectId
0,2021,<NA>,Belle of Louisville,Hospitality Worker,"3,120.00",687.00,0.00,0.00,NaN,687.00,1
1,2021,<NA>,Parks & Recreation,Park Aide,"31,200.00",600.00,0.00,0.00,NaN,600.00,2
2,2021,"Martin, David",Library,Library Page L/U,"28,433.60","28,412.56",0.00,"1,450.00",NaN,"30,862.81",3
3,2021,"Bratcher, Elaine",Louisville Metro Police,Clerk Typist II-Police,"35,256.00","35,256.01",0.00,"1,563.12",NaN,"36,819.13",4
4,2021,"Jackson, Nila",Louisville Metro Police,Traffic Guard II,"21,418.80","16,529.14",952.39,"1,000.00",NaN,"18,481.53",5


- I see some missing values in Name we will fill them. 
- Also checking the other column. 
  - remove if no values. 

In [38]:
sal["Other"].value_counts()

Series([], Name: count, dtype: int64)

Filled other as I was not paying attention to the above and now dropping other. 

In [39]:
sal = sal.fillna({
    "Employee_Name": "Unknown",
    "Other": 0
})

sal = sal.drop("Other", axis=1)
sal.head()

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,ObjectId
0,2021,Unknown,Belle of Louisville,Hospitality Worker,"3,120.00",687.00,0.00,0.00,687.00,1
1,2021,Unknown,Parks & Recreation,Park Aide,"31,200.00",600.00,0.00,0.00,600.00,2
2,2021,"Martin, David",Library,Library Page L/U,"28,433.60","28,412.56",0.00,"1,450.00","30,862.81",3
3,2021,"Bratcher, Elaine",Louisville Metro Police,Clerk Typist II-Police,"35,256.00","35,256.01",0.00,"1,563.12","36,819.13",4
4,2021,"Jackson, Nila",Louisville Metro Police,Traffic Guard II,"21,418.80","16,529.14",952.39,"1,000.00","18,481.53",5


<style>
div.math { text-align: left !important; }
</style>
# Employee Pay and Hours Calculations

This formulas I will use to calculate hourly pay rates, overtime, total hours, and weekly averages for full-time employees.

---

### 1️ Hourly Rate and Minimum Wage Adjustment

Hourly rate assumes 2,080 hours per year (40 hours × 52 weeks):

$$
Hr\_Rate = \frac{Annual\_Rate}{2080}
$$

If the calculated hourly rate is below \$7.25 (U.S. federal minimum wage), it’s adjusted upward:

$$
Hr\_Rate =
\begin{cases}
7.25, & Hr\_Rate < 7.25 \\
Hr\_Rate, & \text{otherwise}
\end{cases}
$$

---

### 2️ Overtime Rate

Overtime is paid at 1.5× the regular hourly rate:

$$
Ot\_Rate = 1.5 \times Hr\_Rate
$$

---

### 3️ Regular Hours and Full-Time Filter

Estimate total regular annual hours:

$$
Reg\_Hours = \frac{Annual\_Rate}{Hr\_Rate}
$$

Filter to include only full-time employees:

$$
Reg\_Hours \ge 2080
$$

---

### 4️ Regular and Overtime Hours Worked

Compute regular and overtime hours based on pay:

$$
Rg\_Hours = \frac{Regular\_Rate}{Hr\_Rate}
$$

$$
Ot\_Hours = \frac{Overtime\_Rate}{Ot\_Rate}
$$

---

### 5️ Total Hours Worked

$$
Total\_Hours = Reg\_Hours + Ot\_Hours
$$

---

### 6️ Weeks Worked

Estimate number of weeks worked:

$$
Weeks\_Worked = \frac{Regular\_Rate}{Hr\_Rate \times 40}
$$

---

### 7️ Weekly Breakdown of Regular and Overtime Hours

$$
Reg\_Hrs\_per\_Week = \frac{Regular\_Rate}{Hr\_Rate \times Weeks\_Worked}
$$

$$
Ot\_Hrs\_per\_Week = \frac{Overtime\_Rate}{Ot\_Rate \times Weeks\_Worked}
$$

---

### 8️ Average Hours per Week

$$
Avg\_Hrs\_per\_Week = Reg\_Hrs\_per\_Week + Ot\_Hrs\_per\_Week
$$




In [40]:
# 1️ Calculate Hourly Rate and apply minimum wage rule
sal["Hr_Rate"] = sal["Annual_Rate"] / 2080 

# If hourly rate is less than $7.25, replace it with $7.25 (minimum wage floor)
sal["Hr_Rate"] = sal["Hr_Rate"].mask(sal["Hr_Rate"] < 7.25, 7.25)

# 2 Calculate Overtime Rate
sal["Ot_Rate"] = sal["Hr_Rate"] * 1.5

# 3️ Filter out part-time employees
sal['Reg_Hours'] = sal["Annual_Rate"] / sal["Hr_Rate"]

# Keep only full-time employees (≥ 2080 hours)
sal = sal[sal['Reg_Hours'] >= 2080]

# 4️ Calculate regular hours worked (based on Regular_Rate)
sal['Rg_Hours'] = sal["Regular_Rate"] / sal["Hr_Rate"]

# 5️ Calculate overtime hours worked
sal['Ot_Hours'] = sal["Overtime_Rate"] / sal["Ot_Rate"]

# 6️ Total hours worked
sal['Total_Hours'] = sal['Reg_Hours'] + sal['Ot_Hours']

# 7️ Estimate number of weeks worked
sal["Weeks_Worked"] = sal["Regular_Rate"] / (sal["Hr_Rate"] * 40) 

# 8️ Regular hours per week
sal["Reg_Hrs_per_Week"] = sal["Regular_Rate"] / (sal["Hr_Rate"] * sal["Weeks_Worked"])

# 9️ Overtime hours per week
sal["Ot_Hrs_per_Week"] = sal["Overtime_Rate"] / (sal["Ot_Rate"] * sal["Weeks_Worked"])

# 10 Average hours per week
sal["Avg_Hrs_per_Week"] = sal["Reg_Hrs_per_Week"] + sal["Ot_Hrs_per_Week"]

sal.head()

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Reg_Hours,Rg_Hours,Ot_Hours,Total_Hours,Weeks_Worked,Reg_Hrs_per_Week,Ot_Hrs_per_Week,Avg_Hrs_per_Week
1,2021,Unknown,Parks & Recreation,Park Aide,"31,200.00",600.00,0.00,0.00,600.00,2,15.00,22.50,"2,080.00",40.00,0.00,"2,080.00",1.00,40.00,0.00,40.00
2,2021,"Martin, David",Library,Library Page L/U,"28,433.60","28,412.56",0.00,"1,450.00","30,862.81",3,13.67,20.51,"2,080.00","2,078.46",0.00,"2,080.00",51.96,40.00,0.00,40.00
3,2021,"Bratcher, Elaine",Louisville Metro Police,Clerk Typist II-Police,"35,256.00","35,256.01",0.00,"1,563.12","36,819.13",4,16.95,25.43,"2,080.00","2,080.00",0.00,"2,080.00",52.00,40.00,0.00,40.00
4,2021,"Jackson, Nila",Louisville Metro Police,Traffic Guard II,"21,418.80","16,529.14",952.39,"1,000.00","18,481.53",5,10.30,15.45,"2,080.00","1,605.16",61.66,"2,141.66",40.13,40.00,1.54,41.54
5,2021,"Ammon, Darrell",Louisville Metro Police,Criminal Justice Specialist,"50,107.20","49,362.47",0.00,0.00,"49,362.47",6,24.09,36.14,"2,080.00","2,049.09",0.00,"2,080.00",51.23,40.00,0.00,40.00


# Keep this file in your project

keep at the bottom of your project

We will go over it later, its a script that will generate a markdown table of contents based on your headings created here in markdown.

In [41]:
import json
import os


def generate_toc_from_notebook(notebook_path):
    """
    Parses a local .ipynb file and generates Markdown for a Table of Contents.
    """
    if not os.path.isfile(notebook_path):
        print(f"❌ Error: File not found at '{notebook_path}'")
        return

    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    toc_markdown = "### **Table of Contents**\n"
    for cell in notebook.get('cells', []):
        if cell.get('cell_type') == 'markdown':
            for line in cell.get('source', []):
                if line.strip().startswith('#'):
                    level = line.count('#')
                    title = line.strip('#').strip()
                    link = title.lower().replace(' ', '-').strip('-.()')
                    indent = '  ' * (level - 1)
                    toc_markdown += f"{indent}* [{title}](#{link})\n"

    print("\n--- ✅ Copy the Markdown below and paste"
          "it into a new markdown cell ---\n")
    print(toc_markdown)


if __name__ == "__main__":
    # Example usage
    notebook_path = 'danny.ipynb'  # Replace with your notebook path
    generate_toc_from_notebook(notebook_path)


--- ✅ Copy the Markdown below and pasteit into a new markdown cell ---

### **Table of Contents**
    * [**Table of Contents**](#**table-of-contents**)
* [Reading in the data](#reading-in-the-data)
  * [Finished import](#finished-import)
* [cleaning](#cleaning)
* [Employee Pay and Hours Calculations](#employee-pay-and-hours-calculations)
    * [1️ Hourly Rate and Minimum Wage Adjustment](#1️-hourly-rate-and-minimum-wage-adjustment)
    * [2️ Overtime Rate](#2️-overtime-rate)
    * [3️ Regular Hours and Full-Time Filter](#3️-regular-hours-and-full-time-filter)
    * [4️ Regular and Overtime Hours Worked](#4️-regular-and-overtime-hours-worked)
    * [5️ Total Hours Worked](#5️-total-hours-worked)
    * [6️ Weeks Worked](#6️-weeks-worked)
    * [7️ Weekly Breakdown of Regular and Overtime Hours](#7️-weekly-breakdown-of-regular-and-overtime-hours)
    * [8️ Average Hours per Week](#8️-average-hours-per-week)
* [Keep this file in your project](#keep-this-file-in-your-project)

